In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import scipy
from tqdm.notebook import tqdm 
from torchvision import transforms, datasets
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torchvision import models
from torchvision.utils import save_image
import os
import gc
import glob

# Download data

In [ ]:
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))
  
# # Then move kaggle.json into the folder where the API expects to find it.
# !mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
data  = pd.read_csv('Pre_Post_Questions.csv', encoding="latin1")

In [ ]:
data.head()

,id,question,item_name,label
0,B017PICGL0,Are boots cold reliable?,"Reebok Work Men's Zigkick RB7005 Work Shoe, Br...",Pre
1,B01HI8YEZS,is the main body rubber or plastic?,"3C-Aone Galaxy S5 Case,Mangix Built-in Glass L...",Pre
2,B07GZ1LF4R,Do you know the thread pitch/count for the tang?,"ColdLand |14.00"" Hand Forged Damascus Steel Bl...",Post
3,B07BMNPRFN,The item ordered came tarnished and my son nee...,BEICHUANG Beidou 7 Stars Big Dipper Star Penda...,Post
4,B075QQKFK3,"""I purchased these and now they look weird wit...",Alla Lighting H8 H11 LED Bulbs Xtreme Super Br...,Post


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231904 entries, 0 to 231903
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         231904 non-null  object
 1   question   231904 non-null  object
 2   item_name  231781 non-null  object
 3   label      231600 non-null  object
dtypes: object(4)
memory usage: 7.1+ MB


In [ ]:
data['label'].value_counts()

Pre                                                                                   133456
Post                                                                                   97674
-1                                                                                        22
0                                                                                          5
 White"                                                                                    5
                                                                                       ...  
101                                                                                        1
 Loop Recording                                                                            1
 Pedometer IP67 Waterproof Sleep Monitor Step Counter for Android & iPhone (Blue)"         1
58                                                                                         1
Motorhomes                                                            

In [ ]:
data = data[data['label'].isin(['Pre', 'Post'])]
data['label'].value_counts()

Pre     133456
Post     97674
Name: label, dtype: int64

In [ ]:
data = data[['question', 'label']]

In [ ]:
data.label.replace({'Pre':0,'Post':1},inplace=True)

In [ ]:
# balance data
index_0 = data[data['label'] == 0].index
data_0 = data[data.index.isin(np.random.choice(index_0, 100000, replace=False))]
data_1 = data[data['label'] == 1]
data = pd.concat([data_0, data_1])
data['label'].value_counts()

0    100000
1     97674
Name: label, dtype: int64

In [ ]:
def find_not_alpha(x):
  return ' '.join(np.unique([i for i in x if not i.isalpha()]))

In [ ]:
import itertools
unique_string_sent = data['question'].apply(lambda x: find_not_alpha(x))
string_not_alpha = np.unique(list(itertools.chain(*unique_string_sent)))

In [ ]:
string_not_alpha

array(['\x08', '\n', '\x16', ' ', '!', '"', '#', '$', '%', '&', "'", '(',
       ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5',
       '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', '[', '\\',
       ']', '^', '_', '`', '{', '|', '}', '~', '\x80', '\x81', '\x82',
       '\x83', '\x84', '\x85', '\x87', '\x88', '\x89', '\x8b', '\x8c',
       '\x8d', '\x8e', '\x8f', '\x90', '\x91', '\x92', '\x93', '\x94',
       '\x95', '\x96', '\x98', '\x99', '\x9a', '\x9b', '\x9c', '\x9d',
       '\x9e', '\x9f', '\xa0', '¡', '¢', '£', '¤', '¥', '¦', '§', '¨',
       '©', '«', '¬', '\xad', '®', '¯', '°', '±', '²', '³', '´', '¶', '·',
       '¸', '¹', '»', '¼', '½', '¾', '¿', '×'], dtype='<U1')

In [ ]:
# def check_string(x, check_char):
index_char = data['question'].apply(lambda x: '¤' in x)
data.loc[index_char]['question']


56950     "When i read it say ""BHB"" salts i thought it...
71504     What is the hem length on these FjÃ¤llrÃ¤ven G...
117863    Are the ocean deep auqa & black? ð¤ð¼ð¤ð...
120651    Does this kill or eliminate the parasites and ...
138551    Black cat t-shirt; eill spandex 5% help it str...
222285    Is it possible to know which dolls we can get?...
227381       Is this comfortable if you wear glasses  ð¤E
82650     Recieved grill however no books were in packin...
89691     Can i buy a replacement charger? I lost mine ð¤·
99921     Why does it say in specs 37.4 inches high... I...
110058    I ordered one last week and it sounded very go...
111505    How do you clean these? I have a pair and my d...
114149    Why does it say set of 2 26x26 and Amazon only...
137344    How do i contact seller?  did not receive pack...
156247    So these are coming from China. Umm i donât ...
158192    This is the battery that was recommended from ...
160929    My bed was supposedly âdeliv

We use this string in the begining

# Simple BASELINE

In [ ]:
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
train_data, test_data, train_target, test_target = train_test_split(data['question'], data['label'], test_size=0.2, random_state=42, shuffle=True, stratify=data['label'])
train_data.head()

168723              "Does this fit a 5.5"" bbl Single Six?"
179164                                         Cracked item
122244    I have an older craftsman 1/2 hp opener that j...
146898    How many leaders do the bags hold? Would it ho...
141099                            What are the ingredients?
Name: question, dtype: object

In [ ]:
train_target

168723    0
179164    1
122244    0
146898    0
141099    0
         ..
87226     0
73885     1
186456    0
231107    1
160828    0
Name: label, Length: 158139, dtype: int64

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=10, max_df=500)

X_train = vectorizer.fit_transform(train_data)
X_test = vectorizer.transform(test_data)

In [ ]:
model = LogisticRegression(solver= 'liblinear')

In [ ]:
model.fit(X_train, train_target)


LogisticRegression(solver='liblinear')

In [ ]:
predicted_train = model.predict(X_train)
predicted_test = model.predict(X_test)

In [ ]:
print(classification_report(train_target, predicted_train))

              precision    recall  f1-score   support

           0       0.66      0.76      0.71     80000
           1       0.71      0.61      0.66     78139

    accuracy                           0.69    158139
   macro avg       0.69      0.68      0.68    158139
weighted avg       0.69      0.69      0.68    158139



In [ ]:
print(classification_report(test_target, predicted_test))

              precision    recall  f1-score   support

           0       0.64      0.74      0.69     20000
           1       0.68      0.57      0.62     19535

    accuracy                           0.66     39535
   macro avg       0.66      0.66      0.65     39535
weighted avg       0.66      0.66      0.66     39535



In [ ]:
data.to_csv('pre_post_question_preparing.csv', index=False)